## test on pic

In [65]:
import cv2
from ultralytics import YOLO
import torch
import numpy as np
import arabic_reshaper
from bidi.algorithm import get_display
from PIL import Image, ImageDraw, ImageFont
import pyautogui  # 📌 To get screen size dynamically

# Mapping class IDs to Arabic characters
id2char = {
    0: 'أ', 1: 'ب', 2: 'ج', 3: 'د', 4: 'ر', 5: 'س', 6: 'ص', 7: 'ط', 8: 'ع', 9: 'ف',
    10: 'ق', 11: 'ل', 12: 'م', 13: 'ن', 14: 'ھ', 15: 'و', 16: 'ى',
    17: '٠', 18: '١', 19: '٢', 20: '٣', 21: '٤', 22: '٥', 23: '٦', 24: '٧', 25: '٨', 26: '٩'
}

# Load YOLO models
plate_model = YOLO("best.pt")       # Replace with your trained plate detector
char_model = YOLO("chars.pt")       # Replace with your trained character detector

# Load and prepare the image
img = cv2.imread("o.jpg")
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Step 1: Detect license plate
plate_results = plate_model(img_rgb)[0]
if len(plate_results.boxes) == 0:
    print("❌ No plate found")
    exit()

# Use the first detected plate
plate_box = plate_results.boxes[0].xyxy[0].cpu().numpy().astype(int)
x1, y1, x2, y2 = plate_box
plate_crop = img[y1:y2, x1:x2]

# Step 2: Detect characters on the cropped plate
char_results = char_model(plate_crop)[0]
char_boxes = char_results.boxes

if len(char_boxes) == 0:
    print("❌ No characters found on plate")
    exit()

digit_indices = set(range(17, 27))
letters = []
digits = []

# Draw bounding boxes on cropped plate (no text)
for box, cls in zip(char_boxes.xyxy, char_boxes.cls):
    x1_c, y1_c, x2_c, y2_c = map(int, box)
    x_center = (x1_c + x2_c) / 2
    char = id2char[int(cls)]

    cv2.rectangle(plate_crop, (x1_c, y1_c), (x2_c, y2_c), (0, 255, 0), 2)

    if int(cls) in digit_indices:
        digits.append((x_center, char))
    else:
        letters.append((x_center, char))

# Sort characters
letters.sort(key=lambda x: x[0])  # RTL
digits.sort(key=lambda x: x[0])   # LTR

# Compose final plate text
arabic_letters = ' '.join([char for _, char in reversed(letters)])  # RTL
arabic_digits = ''.join([char for _, char in digits])
final_plate = arabic_letters + ' ' + arabic_digits

print("✅ Detected Plate Text:", final_plate)

# ===== Arabic Reshaping and Text Rendering =====
reshaped_text = arabic_reshaper.reshape(final_plate)
bidi_text = get_display(reshaped_text)

# Convert image to PIL
img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
draw = ImageDraw.Draw(img_pil)

# Load Arabic-supporting font
try:
    font = ImageFont.truetype("arial.ttf", 40)  # Windows
except:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 40)  # Linux

# Draw Arabic text above the plate
draw.text((x1, y1 - 50), bidi_text, font=font, fill=(255, 0, 0))
draw.rectangle([x1, y1, x2, y2], outline=(0, 0, 255), width=4)

# Convert back to OpenCV format
img_annotated = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# ===== 📌 Resize for Display Based on Screen Size =====
screen_width, screen_height = pyautogui.size()
max_display_width = int(screen_width * 0.9)
max_display_height = int(screen_height * 0.9)

h, w = img_annotated.shape[:2]
scale_w = max_display_width / w
scale_h = max_display_height / h
scale = min(scale_w, scale_h, 1.0)  # Do not upscale

new_w = int(w * scale)
new_h = int(h * scale)
resized_img = cv2.resize(img_annotated, (new_w, new_h))

# ===== Show and Save Result =====
cv2.imshow("Detected Plate", resized_img)
cv2.imwrite("annotated_plate.jpg", img_annotated)  # Save full resolution
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 800x704 1 License Plate, 31.3ms
Speed: 6.6ms preprocess, 31.3ms inference, 2.1ms postprocess per image at shape (1, 3, 800, 704)

0: 320x640 1 , 1 , 1 , 4 s, 16.8ms
Speed: 1.3ms preprocess, 16.8ms inference, 3.1ms postprocess per image at shape (1, 3, 320, 640)
✅ Detected Plate Text: ر أ م ١١١١


## test video


In [56]:
import cv2
from ultralytics import YOLO
import torch
import numpy as np
import arabic_reshaper
from bidi.algorithm import get_display
from PIL import Image, ImageDraw, ImageFont
import pyautogui  # For screen size

# Arabic character map
id2char = {
    0: 'أ', 1: 'ب', 2: 'ج', 3: 'د', 4: 'ر', 5: 'س', 6: 'ص', 7: 'ط', 8: 'ع', 9: 'ف',
    10: 'ق', 11: 'ل', 12: 'م', 13: 'ن', 14: 'ھ', 15: 'و', 16: 'ى',
    17: '٠', 18: '١', 19: '٢', 20: '٣', 21: '٤', 22: '٥', 23: '٦', 24: '٧', 25: '٨', 26: '٩'
}

# Load models
plate_model = YOLO("best.pt")
char_model = YOLO("chars.pt")

# Font loading (adjust path if needed)
try:
    font = ImageFont.truetype("arial.ttf", 40)  # Windows
except:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 40)  # Linux

# Screen size
screen_width, screen_height = pyautogui.size()

# === VIDEO SETUP ===
video_path = "testvideo.mp4"  # 0 = webcam, or replace with "video.mp4"
cap = cv2.VideoCapture(video_path)

# Optional: save output
save_output = False
if save_output:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter("output_video.mp4", fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plate_results = plate_model(img_rgb)[0]

    img_pil = Image.fromarray(img_rgb)
    draw = ImageDraw.Draw(img_pil)

    for plate_box in plate_results.boxes:
        x1, y1, x2, y2 = plate_box.xyxy[0].cpu().numpy().astype(int)
        plate_crop = frame[y1:y2, x1:x2]
        char_results = char_model(plate_crop)[0]
        char_boxes = char_results.boxes

        if len(char_boxes) == 0:
            continue

        letters, digits = [], []
        for box, cls in zip(char_boxes.xyxy, char_boxes.cls):
            x1_c, y1_c, x2_c, y2_c = map(int, box)
            x_center = (x1_c + x2_c) / 2
            char = id2char[int(cls)]

            if int(cls) in range(17, 27):  # Digits
                digits.append((x_center, char))
            else:  # Letters
                letters.append((x_center, char))

        letters.sort(key=lambda x: x[0])
        digits.sort(key=lambda x: x[0])
        arabic_letters = ' '.join([char for _, char in reversed(letters)])
        arabic_digits = ''.join([char for _, char in digits])
        final_plate = arabic_letters + ' ' + arabic_digits

        reshaped_text = arabic_reshaper.reshape(final_plate)
        bidi_text = get_display(reshaped_text)

        draw.text((x1, y1 - 50), bidi_text, font=font, fill=(255, 0, 0))
        draw.rectangle([x1, y1, x2, y2], outline=(0, 255, 0), width=3)

    # Convert back to OpenCV image
    annotated_frame = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

    # Auto resize to fit screen if necessary
    frame_h, frame_w = annotated_frame.shape[:2]
    if frame_w > screen_width or frame_h > screen_height:
        scale = min(screen_width / frame_w, screen_height / frame_h)
        resized_frame = cv2.resize(annotated_frame, (int(frame_w * scale), int(frame_h * scale)))
    else:
        resized_frame = annotated_frame.copy()

    # Show result
    cv2.imshow("Arabic Plate Detection", resized_frame)

    if save_output:
        out.write(annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
if save_output:
    out.release()
cv2.destroyAllWindows()



0: 800x480 (no detections), 78.6ms
Speed: 14.3ms preprocess, 78.6ms inference, 2.7ms postprocess per image at shape (1, 3, 800, 480)

0: 800x480 (no detections), 27.6ms
Speed: 6.8ms preprocess, 27.6ms inference, 2.1ms postprocess per image at shape (1, 3, 800, 480)

0: 800x480 (no detections), 31.9ms
Speed: 8.5ms preprocess, 31.9ms inference, 1.9ms postprocess per image at shape (1, 3, 800, 480)

0: 800x480 (no detections), 33.5ms
Speed: 11.0ms preprocess, 33.5ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 480)

0: 800x480 (no detections), 34.2ms
Speed: 11.8ms preprocess, 34.2ms inference, 2.1ms postprocess per image at shape (1, 3, 800, 480)

0: 800x480 (no detections), 23.5ms
Speed: 11.4ms preprocess, 23.5ms inference, 1.3ms postprocess per image at shape (1, 3, 800, 480)

0: 800x480 (no detections), 35.5ms
Speed: 8.1ms preprocess, 35.5ms inference, 2.4ms postprocess per image at shape (1, 3, 800, 480)

0: 800x480 (no detections), 39.1ms
Speed: 10.8ms preprocess, 39.